# Circuit Decomposition 
Edge and gate decomposition with zero classical communication.

Prerequisites: 

1. Create a new Conda environment
```shell
conda create --name azure-knitting python=3.9.17
conda activate azure-knitting
```

1. Install everything needed to run the `modified-circuit-knitting-toolbox` locally: 
```shell
cd modified-circuit-knitting-toolbox
python -m pip install -e .
python -m pip install 'docplex>=2.23.222' 'cplex>=22.1.0.0' 'gurobipy==11.0.0'
```

1. Install everything needed to run on Azure:
```shell
python -m pip install --upgrade 'azure-quantum[qiskit]==1.2.1'
# python -m pip install qiskit-aer==0.12.1
```

### Documentation
Running on Azure: [instructions](https://learn.microsoft.com/en-us/azure/quantum/quickstart-microsoft-qiskit?tabs=tabid-quantinuum&pivots=platform-local#prerequisites)


First, enable communication with the instance

In [1]:
# Prevent from overriding already run experiments
exit()

In [1]:
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider


workspace = Workspace(
            resource_id = "/subscriptions/4ed1f7fd-7d9e-4c61-9fac-521649937e65/resourceGroups/Cutting/providers/Microsoft.Quantum/Workspaces/Cutting",
            location = "eastus")


provider = AzureQuantumProvider(workspace)

Check what Azure targets are available. Note that Quantinuum may need to be enabled in your workspace.

In [2]:
print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

This workspace's targets:
- quantinuum.sim.h1-1sc
- quantinuum.sim.h1-1sc
- quantinuum.sim.h1-1e
- quantinuum.sim.h1-1e
- quantinuum.qpu.h1-1
- quantinuum.qpu.h1-1


In [3]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.circuit.library import EfficientSU2

# Create a quantum circuit to cut. We create a simple ansatz
mapper = JordanWignerMapper()
ansatz = EfficientSU2(3, reps=1)
# Decompose to the actual individual gates
circuit = ansatz.decompose(reps=3)
# Set some arbitrary parameters
circuit.assign_parameters([0.8] * len(circuit.parameters), inplace=True)

print(circuit)

global phase: 3.8832
     ┌─────────────┐┌─────────────┐                    ┌─────────────┐»
q_0: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├────────────■───────┤ U3(0.8,0,0) ├»
     ├─────────────┤├─────────────┤          ┌─┴─┐     ├─────────────┤»
q_1: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├──■───────┤ X ├─────┤ U3(0.8,0,0) ├»
     ├─────────────┤├─────────────┤┌─┴─┐┌────┴───┴────┐├─────────────┤»
q_2: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ X ├┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├»
     └─────────────┘└─────────────┘└───┘└─────────────┘└─────────────┘»
«     ┌─────────────┐
«q_0: ┤ U3(0,0,0.8) ├
«     ├─────────────┤
«q_1: ┤ U3(0,0,0.8) ├
«     └─────────────┘
«q_2: ───────────────
«                    


Create a list of observables to use, and automatically find cut locations.

In [5]:
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import cut_wires_and_gates_to_subcircuits

observables = ["ZZI", "IZZ", "IIZ", "XIX", "ZIZ", "IXI"]

subcircuits, subobservables, _, _ = cut_wires_and_gates_to_subcircuits(
    circuit=circuit,
    observables=observables,
    method='automatic',
    max_subcircuit_width=2,
    max_cuts=9,
    num_subcircuits=[2]
)

MIP MODEL CUT WIRES:  []


In [6]:
# Visualize the subcircuits. Note the decomposed 2-qubit gates marked 'cut_cx_0'
for key in subcircuits.keys():
    print(subcircuits[key])

      ┌─────────────┐┌─────────────┐                 ┌─────────────┐»
q5_0: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├──────────────■──┤ U3(0.8,0,0) ├»
      ├─────────────┤├─────────────┤┌──────────┐┌─┴─┐├─────────────┤»
q5_1: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ cut_cx_0 ├┤ X ├┤ U3(0.8,0,0) ├»
      └─────────────┘└─────────────┘└──────────┘└───┘└─────────────┘»
«      ┌─────────────┐
«q5_0: ┤ U3(0,0,0.8) ├
«      ├─────────────┤
«q5_1: ┤ U3(0,0,0.8) ├
«      └─────────────┘
    ┌─────────────┐┌─────────────┐┌──────────┐┌─────────────┐┌─────────────┐
q6: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ cut_cx_0 ├┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├
    └─────────────┘└─────────────┘└──────────┘└─────────────┘└─────────────┘


In [7]:
from circuit_knitting.cutting.gate_and_wire_cutting.evaluation import azure_queue_experiments


# Use the Quantinuum emulator backend
# FIXME: use syntax checker for early testing
backend_str = 'quantinuum.sim.h1-1e'
backend = provider.get_backend(backend_str)

# Submit the subcircuits to Azure Quantum
job_list, qpd_list, coefficients, subexperiments = azure_queue_experiments(
    circuits=subcircuits,
    subobservables=subobservables,
    num_samples=8,  # 8 unique samples to get some statistics
    backend = backend,
    # provider = provider,
    shots=128  # Balance of cost and accuracy
)

Check on the actual results from run on Quantinuum.

In [8]:
from circuit_knitting.cutting.gate_and_wire_cutting.evaluation import get_experiment_results_from_jobs

experiment_results = get_experiment_results_from_jobs(job_list, qpd_list, coefficients)
experiment_results

............

CuttingExperimentResults(quasi_dists=[[[({0: 0.34375, 1: 0.1015625, 2: 0.28125, 3: 0.2734375}, 0), ({2: 0.0625, 0: 0.5234375, 3: 0.0546875, 1: 0.359375}, 0)], [({0: 0.0859375, 1: 0.9140625}, 0), ({0: 0.28125, 1: 0.71875}, 0)]], [[({0: 0.84375, 1: 0.0703125, 3: 0.0859375}, 0), ({2: 0.578125, 0: 0.171875, 3: 0.0234375, 1: 0.2265625}, 0)], [({0: 0.4921875, 1: 0.5078125}, 0), ({0: 0.9921875, 1: 0.0078125}, 0)]], [[({5: 0.09375, 2: 0.078125, 6: 0.15625, 4: 0.1015625, 0: 0.5625, 7: 0.0078125}, 1), ({5: 0.0546875, 2: 0.21875, 4: 0.3046875, 0: 0.34375, 1: 0.015625, 3: 0.0078125, 7: 0.0546875}, 1)], [({0: 0.5625, 1: 0.4375}, 0), ({0: 0.6328125, 1: 0.3671875}, 0)]], [[({5: 0.078125, 2: 0.0546875, 6: 0.1796875, 4: 0.0703125, 0: 0.5703125, 1: 0.015625, 3: 0.0078125, 7: 0.0234375}, 1), ({5: 0.0546875, 2: 0.203125, 6: 0.015625, 4: 0.265625, 0: 0.359375, 1: 0.015625, 7: 0.0859375}, 1)], [({0: 0.0625, 1: 0.9375}, 0), ({0: 0.671875, 1: 0.328125}, 0)]], [[({0: 0.4765625, 1: 0.0390625, 2: 0.2578125, 3: 0

Now check the actual expectation values from the observables.

In [9]:
from circuit_knitting.cutting.cutting_reconstruction import reconstruct_expectation_values

quantinuum_expvals = reconstruct_expectation_values(*experiment_results, subobservables)
quantinuum_expvals

[0.40625,
 0.4658203125,
 0.399169921875,
 0.1591796875,
 0.130126953125,
 0.27294921875]

In [10]:
# Create ideal results
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import exact_observables

ideal_expvals = exact_observables(circuit, observables)
ideal_expvals

array([0.50390696, 0.57454157, 0.39584538, 0.09798816, 0.18481229,
       0.23530298])

In [11]:
# Compare the error between results
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import compare_results

compare_results(quantinuum_expvals, ideal_expvals)

Simulated expectation values: [0.40625, 0.46582031, 0.39916992, 0.15917969, 0.13012695, 0.27294922]
Exact expectation values: [0.50390696, 0.57454157, 0.39584538, 0.09798816, 0.18481229, 0.23530298]
Errors in estimation: [-0.09765696, -0.10872126, 0.00332454, 0.06119152, -0.05468534, 0.03764624]
Relative errors in estimation: [-0.19379959, -0.18923132, 0.00839858, 0.62447874, -0.29589666, 0.15999051]


([-0.09765696, -0.10872126, 0.00332454, 0.06119152, -0.05468534, 0.03764624],
 [-0.19379959, -0.18923132, 0.00839858, 0.62447874, -0.29589666, 0.15999051])

The below is a cost estimator for the circuits run - make sure to add up for all subcircuits run.

In [12]:
cost_backend = provider.get_backend('quantinuum.qpu.h1-1')
total_cost = 0
for sample in subexperiments:
    for partition in sample:
        for subexperiment in partition:
            cost = cost_backend.estimate_cost(subexperiment, shots=128)
            total_cost += cost.estimated_total
print(f'Estimated cost: {total_cost} {cost.currency_code}')

Estimated cost: 133.8752 HQC
